参考https://qiita.com/haru1977/items/17833e508fe07c004119

In [4]:
import os
import cv2
from skimage.filters import sobel_h, sobel_v, sobel
from skimage.feature import canny
import numpy as np
import sys

RUN_ID = 5

In [5]:
def load_images(dirname, gray = False, normalization = True, flip_augment = True, chanel_extension = False, crop = True, detect = True):
    imlist =[]
    for filename in os.listdir(dirname):
        im = np.array(cv2.imread(dirname+filename))
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        
        if detect is True:
#             size_y = 400
            size_y=300
#             print(size_y)
            im_filter = canny(im_gray)
            im_filter = np.array(im_filter, dtype=np.uint8)
            M = cv2.moments(im_filter)
            center_x = int(M['m10']/M['m00'])
            center_y = int(M['m01']/M['m00'])
#             im = im[int(center_y - size_y/2) : int(center_y + size_y/2), 500:-200]
            im = im[int(center_y - size_y/2) : int(center_y + size_y/2), 100:-50]
#             im_gray = im_gray[int(center_y - size_y/2) : int(center_y + size_y/2), 500:-200]
            im_gray = im_gray[int(center_y - size_y/2) : int(center_y + size_y/2), 200:-50]
        if crop is True:
            im = im[10:-10, 90:-90]
            im_gray = im_gray[10:-10, 90:-90]

        if gray is True:
            im = im_gray

        if normalization is True:
            im = (im - np.average(im))/np.std(im)

        if chanel_extension is "canny":
            if gray is True:
                im_canny = np.resize(canny(im), (im.shape[0], im.shape[1], 1))
                im = np.resize(im, (im.shape[0], im.shape[1], 1))

            if gray is False:
                im_canny = np.resize(canny(im_gray), (im.shape[0], im.shape[1], 1))

            im = np.concatenate((im, im_canny), axis=2)

        if chanel_extension is "sobel_vh":
            if gray is True:
                im_sobel_h = np.resize(sobel_h(im), (im.shape[0], im.shape[1], 1))
                im_sobel_v = np.resize(sobel_v(im), (im.shape[0], im.shape[1], 1))
                im = np.resize(im, (im.shape[0], im.shape[1], 1))

            if gray is False:
                im_sobel_h = np.resize(sobel_h(im_gray), (im.shape[0], im.shape[1], 1))
                im_sobel_v = np.resize(sobel_v(im_gray), (im.shape[0], im.shape[1], 1))

            im = np.concatenate((im, im_sobel_h, im_sobel_v), axis=2)

        if chanel_extension is "sobel":
            if gray is True:
                im_sobel = np.resize(sobel(im), (im.shape[0], im.shape[1], 1))
                im = np.resize(im, (im.shape[0], im.shape[1], 1))

            if gray is False:
                im_sobel = np.resize(sobel(im_gray), (im.shape[0], im.shape[1], 1))

            im = np.concatenate((im, im_sobel), axis=2)

        if flip_augment is True:
            imlist.append(np.fliplr(im))
            imlist.append(np.flipud(im))
            imlist.append(np.fliplr(np.flipud(im)))

        imlist.append(im)

    imlist = np.array(imlist)
#     print(imlist.shape)
    return imlist

In [6]:
if __name__ == "__main__":
    imlist_ok = load_images("data/sd_ok/", gray = False, normalization = True, flip_augment = True, chanel_extension = "canny", crop = True, detect = False)
    imlist_ng = load_images("data/sd_ng/", gray = False, normalization = True, flip_augment = True, chanel_extension = "canny", crop = True, detect = False)

#     print(imlist_ok.shape, imlist_ng_train.shape, imlist_ng_valid.shape, imlist_ok_test.shape, imlist_ng_test.shape)
    print("OKの画像数："+str(imlist_ok.shape[0]), "NGの画像数"+str(imlist_ng.shape[0]), "合計"+str(imlist_ok.shape[0]+imlist_ng.shape[0]))
    
#     :はスライス
    x_train = np.concatenate([imlist_ok[:int(imlist_ok.shape[0]*0.8)], imlist_ng[:int(imlist_ng.shape[0]*0.8)]], axis=0)    
    
    x_valid = np.concatenate( [imlist_ok[int(imlist_ok.shape[0]*0.8):],imlist_ng[int(imlist_ng.shape[0]*0.8):]], axis=0)
#後ろから読み込んではいけない。trainとvalidで読み込む画像が重なる可能性があるため
    x_test = np.concatenate( [imlist_ok[int(imlist_ok.shape[0]*0.8):],imlist_ng[int(imlist_ng.shape[0]*0.8):]], axis=0)
    
    y_train = np.array([0]*int(imlist_ok[:int(imlist_ok.shape[0]*0.8)].shape[0]) + [1]*int(imlist_ng[:int(imlist_ng.shape[0]*0.8)].shape[0]))
    
    y_valid = np.array([0]*int(imlist_ok[int(imlist_ok.shape[0]*0.8):].shape[0]) + [1]*int(imlist_ng[int(imlist_ng.shape[0]*0.8):].shape[0]))
    
    y_test = np.array([0]*int(imlist_ok[int(imlist_ok.shape[0]*0.8):].shape[0]) + [1]*int(imlist_ng[int(imlist_ng.shape[0]*0.8):].shape[0]))
    
    print("x_train:{}, x_valid:{}, x_test:{}".format(x_train.shape, x_valid.shape, x_test.shape))
    print("y_train:{}, y_valid:{}, y_test:{}".format(y_train.shape, y_valid.shape, y_test.shape))
#     np.save("npy/x_train{}".format(sys.argv[1]), x_train)
    np.save("npy/x_train{}".format(RUN_ID), x_train)
    np.save("npy/x_valid{}".format(RUN_ID), x_valid)
    np.save("npy/x_test{}".format(RUN_ID), x_test)
    np.save("npy/y_train{}".format(RUN_ID), y_train)
    np.save("npy/y_valid{}".format(RUN_ID), y_valid)
    np.save("npy/y_test{}".format(RUN_ID), y_test)


OKの画像数：116 NGの画像数192 合計308
x_train:(245, 460, 460, 4), x_valid:(63, 460, 460, 4), x_test:(63, 460, 460, 4)
y_train:(245,), y_valid:(63,), y_test:(63,)
